In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_KmerRFE"
outPath = "Results"
foldName = "folds.pickle"

shuffle = True
seed = None

input_data_folder = "PredNTS_MathFeature_ENC"

monitor = 'val_loss'

sub_feature_count = 500

In [2]:
train_data_filename = 'Training-datasets-PredNTS_kmer.csv'
indpe_data_filename = 'independent-dataset-PredNTS_kmer.csv'

In [3]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report, matthews_corrcoef

from sklearn.feature_selection import RFE

import math

In [4]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# ##################################################################################
# ##### define all CUSTOM functions
# ##################################################################################

# def one_hot_encode_nt(sequence, char_dict):
    
#     seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
#     i = 0
#     for single_character in sequence:
#         if(single_character.upper() in char_dict.keys()):
#             seq_encoded[i][char_dict[single_character.upper()]] = 1
#             i = i+1
#         else:
#             raise ValueError('Incorrect character in NT sequence: '+sequence)
#     return seq_encoded

In [6]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [7]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [8]:
# epochs = 200
# batch_size = 32

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 2048, ## Dense layer parameters,
#                     dense_layers = 5,
#                     prob = 0.6, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)

#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [9]:
# epochs = 200
# batch_size = 32

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 16, ## Dense layer parameters,
#                     dense_layers = 3,
#                     prob = 0.5, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.BatchNormalization()(y)
#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [10]:
# epochs = 200
# batch_size = 16

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 8, ## Dense layer parameters,
#                     dense_layers = 2,
#                     prob = 0.5, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.BatchNormalization()(y)
#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [11]:
# epochs = 200
# batch_size = 16

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 8, ## Dense layer parameters,
#                     dense_layers = 2,
#                     prob_input = 0.9, prob = 0.5,
#                     learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dropout(prob_input)(input1)
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(y)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.BatchNormalization()(y)
#         y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [12]:
epochs = 200
batch_size = 16

##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 128, ## Dense layer parameters
                    prob = 0.5, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/2), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/4), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [13]:
DLNN_Classifier((sub_feature_count)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500)]             0         
                                                                 
 dense (Dense)               (None, 128)               64128     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0     

# Train data preparation

In [14]:
##################################################################################
##### Read CSV data
##################################################################################

train_data_filepath = os.path.join(input_data_folder, train_data_filename)
train_data = pd.read_csv(train_data_filepath, sep=',', header=0)
train_data = train_data.drop('label', axis=1)

train_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in train_data['nameseq']])

train_data = train_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = np.array(train_data.drop('label', axis=1))
train_labels = np.array(train_data['label'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

In [15]:
##################################################################################
##### Recursive feature selection
##################################################################################

model = DecisionTreeClassifier(criterion="gini")

selector = RFE(model, n_features_to_select=sub_feature_count, step=50)
selector = selector.fit(train_features, train_labels)

feature_indices = np.where(selector.ranking_ == 1)[0]

In [16]:
##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = train_features[:, feature_indices]

folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

input_vec_shape = train_features[0].shape

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Independent data

In [17]:
##################################################################################
##### Read CSV data
##################################################################################

indpe_data_filepath = os.path.join(input_data_folder, indpe_data_filename)
indpe_data = pd.read_csv(indpe_data_filepath, sep=',', header=0)
indpe_data = indpe_data.drop('label', axis=1)

indpe_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in indpe_data['nameseq']])

indpe_data = indpe_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(indpe_data.drop('label', axis=1))
indpe_features = indpe_features[:, feature_indices]

indpe_labels = np.array(indpe_data['label'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

# Training

In [18]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = monitor, verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()


Train/Test model on Fold #0.
Epoch 1/200
109/120 [==========================>...] - ETA: 0s - loss: 0.9992 - accuracy: 0.5218
Epoch 1: val_loss improved from inf to 0.97105, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KmerRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 2s 6ms/step - loss: 0.9973 - accuracy: 0.5228 - val_loss: 0.9710 - val_accuracy: 0.5660
Epoch 2/200
108/120 [==========================>...] - ETA: 0s - loss: 0.9522 - accuracy: 0.5353
Epoch 2: val_loss improved from 0.97105 to 0.92804, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KmerRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 5ms/step - loss: 0.9503 - accuracy: 0.5381 - val_loss: 0.9280 - val_accuracy: 0.6310
Epoch 3/200
112/120 [===========================>..] - ETA: 0s - loss: 0.9122 - accuracy: 0.5357
Epoch 3: val_loss improved from 0.92804 to 0.89303, saving model to Results\NT_Site_PredNTS_Classification_DLNN_K

120/120 [==============================] - 1s 4ms/step - loss: 0.6166 - accuracy: 0.7522 - val_loss: 0.6310 - val_accuracy: 0.7358
Epoch 23/200
119/120 [============================>.] - ETA: 0s - loss: 0.6109 - accuracy: 0.7568
Epoch 23: val_loss improved from 0.63098 to 0.62700, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KmerRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6107 - accuracy: 0.7570 - val_loss: 0.6270 - val_accuracy: 0.7317
Epoch 24/200
106/120 [=========================>....] - ETA: 0s - loss: 0.6191 - accuracy: 0.7353
Epoch 24: val_loss improved from 0.62700 to 0.62466, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KmerRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6122 - accuracy: 0.7412 - val_loss: 0.6247 - val_accuracy: 0.7317
Epoch 25/200
117/120 [============================>.] - ETA: 0s - loss: 0.5969 - accuracy: 0.7634
Epo

120/120 [==============================] - 1s 4ms/step - loss: 0.5493 - accuracy: 0.8131 - val_loss: 0.6152 - val_accuracy: 0.7421
Epoch 48/200
114/120 [===========================>..] - ETA: 0s - loss: 0.5508 - accuracy: 0.7988
Epoch 48: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.5534 - accuracy: 0.8000 - val_loss: 0.6152 - val_accuracy: 0.7463
Epoch 49/200
112/120 [===========================>..] - ETA: 0s - loss: 0.5396 - accuracy: 0.8047
Epoch 49: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.5409 - accuracy: 0.8031 - val_loss: 0.6157 - val_accuracy: 0.7463
Epoch 50/200
112/120 [===========================>..] - ETA: 0s - loss: 0.5385 - accuracy: 0.8019
Epoch 50: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.5369 - accuracy: 0.8021 - val_loss: 0.6162 - val_accuracy: 0.7421
Epoch 51/200
114/120 [================

116/120 [============================>.] - ETA: 0s - loss: 0.5052 - accuracy: 0.8351
Epoch 77: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.5077 - accuracy: 0.8336 - val_loss: 0.6288 - val_accuracy: 0.7484
Epoch 78/200
110/120 [==========================>...] - ETA: 0s - loss: 0.5210 - accuracy: 0.8239
Epoch 78: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.5196 - accuracy: 0.8231 - val_loss: 0.6275 - val_accuracy: 0.7505
Epoch 79/200
114/120 [===========================>..] - ETA: 0s - loss: 0.5177 - accuracy: 0.8158
Epoch 79: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.5188 - accuracy: 0.8168 - val_loss: 0.6277 - val_accuracy: 0.7547
Epoch 80/200
114/120 [===========================>..] - ETA: 0s - loss: 0.5155 - accuracy: 0.8213
Epoch 80: val_loss did not improve from 0.61439
120/120 [===========================

Epoch 107/200
112/120 [===========================>..] - ETA: 0s - loss: 0.4898 - accuracy: 0.8382
Epoch 107: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.4901 - accuracy: 0.8367 - val_loss: 0.6436 - val_accuracy: 0.7400
Epoch 108/200
114/120 [===========================>..] - ETA: 0s - loss: 0.4953 - accuracy: 0.8410
Epoch 108: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.4940 - accuracy: 0.8420 - val_loss: 0.6418 - val_accuracy: 0.7463
Epoch 109/200
114/120 [===========================>..] - ETA: 0s - loss: 0.4872 - accuracy: 0.8377
Epoch 109: val_loss did not improve from 0.61439
120/120 [==============================] - 1s 4ms/step - loss: 0.4899 - accuracy: 0.8373 - val_loss: 0.6429 - val_accuracy: 0.7463
Epoch 110/200
113/120 [===========================>..] - ETA: 0s - loss: 0.4937 - accuracy: 0.8385
Epoch 110: val_loss did not improve from 0.61439
120/120 [======

...

109/120 [==========================>...] - ETA: 0s - loss: 0.5228 - accuracy: 0.8177
Epoch 58: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.5263 - accuracy: 0.8148 - val_loss: 0.6159 - val_accuracy: 0.7269
Epoch 59/200
108/120 [==========================>...] - ETA: 0s - loss: 0.5306 - accuracy: 0.8177
Epoch 59: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.5294 - accuracy: 0.8195 - val_loss: 0.6153 - val_accuracy: 0.7290
Epoch 60/200
110/120 [==========================>...] - ETA: 0s - loss: 0.5197 - accuracy: 0.8278
Epoch 60: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.5217 - accuracy: 0.8237 - val_loss: 0.6163 - val_accuracy: 0.7311
Epoch 61/200
109/120 [==========================>...] - ETA: 0s - loss: 0.5284 - accuracy: 0.8268
Epoch 61: val_loss did not improve from 0.59550
120/120 [===========================

Epoch 88/200
108/120 [==========================>...] - ETA: 0s - loss: 0.5023 - accuracy: 0.8345
Epoch 88: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.5058 - accuracy: 0.8332 - val_loss: 0.6402 - val_accuracy: 0.7248
Epoch 89/200
109/120 [==========================>...] - ETA: 0s - loss: 0.4993 - accuracy: 0.8394
Epoch 89: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.4997 - accuracy: 0.8389 - val_loss: 0.6398 - val_accuracy: 0.7374
Epoch 90/200
107/120 [=========================>....] - ETA: 0s - loss: 0.4948 - accuracy: 0.8376
Epoch 90: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.4977 - accuracy: 0.8379 - val_loss: 0.6411 - val_accuracy: 0.7374
Epoch 91/200
109/120 [==========================>...] - ETA: 0s - loss: 0.5086 - accuracy: 0.8297
Epoch 91: val_loss did not improve from 0.59550
120/120 [==============

120/120 [==============================] - 1s 4ms/step - loss: 0.4892 - accuracy: 0.8389 - val_loss: 0.6593 - val_accuracy: 0.7290
Epoch 118/200
105/120 [=========================>....] - ETA: 0s - loss: 0.4713 - accuracy: 0.8577
Epoch 118: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.4838 - accuracy: 0.8505 - val_loss: 0.6621 - val_accuracy: 0.7290
Epoch 119/200
112/120 [===========================>..] - ETA: 0s - loss: 0.4801 - accuracy: 0.8577
Epoch 119: val_loss did not improve from 0.59550
120/120 [==============================] - 1s 4ms/step - loss: 0.4821 - accuracy: 0.8557 - val_loss: 0.6609 - val_accuracy: 0.7269
Epoch 120/200
107/120 [=========================>....] - ETA: 0s - loss: 0.4885 - accuracy: 0.8575
Epoch 120: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.4909 - accuracy: 0.8541 - val_loss: 0.6627 - val_accuracy: 0.7311
Epoch 121/200
109/120 [=========

Epoch 147/200
106/120 [=========================>....] - ETA: 0s - loss: 0.4612 - accuracy: 0.8626
Epoch 147: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.4690 - accuracy: 0.8547 - val_loss: 0.6727 - val_accuracy: 0.7332
Epoch 148/200
108/120 [==========================>...] - ETA: 0s - loss: 0.4678 - accuracy: 0.8559
Epoch 148: val_loss did not improve from 0.59550
120/120 [==============================] - 1s 4ms/step - loss: 0.4709 - accuracy: 0.8547 - val_loss: 0.6732 - val_accuracy: 0.7353
Epoch 149/200
105/120 [=========================>....] - ETA: 0s - loss: 0.4839 - accuracy: 0.8595
Epoch 149: val_loss did not improve from 0.59550
120/120 [==============================] - 0s 4ms/step - loss: 0.4784 - accuracy: 0.8589 - val_loss: 0.6841 - val_accuracy: 0.7143
Epoch 150/200
105/120 [=========================>....] - ETA: 0s - loss: 0.4810 - accuracy: 0.8524
Epoch 150: val_loss did not improve from 0.59550
120/120 [======

120/120 [==============================] - 1s 5ms/step - loss: 0.4789 - accuracy: 0.8583 - val_loss: 0.6862 - val_accuracy: 0.7311
Epoch 177/200
114/120 [===========================>..] - ETA: 0s - loss: 0.4739 - accuracy: 0.8558
Epoch 177: val_loss did not improve from 0.59550
120/120 [==============================] - 1s 6ms/step - loss: 0.4725 - accuracy: 0.8583 - val_loss: 0.6877 - val_accuracy: 0.7290
Epoch 178/200
115/120 [===========================>..] - ETA: 0s - loss: 0.4638 - accuracy: 0.8576
Epoch 178: val_loss did not improve from 0.59550
120/120 [==============================] - 1s 5ms/step - loss: 0.4643 - accuracy: 0.8573 - val_loss: 0.6939 - val_accuracy: 0.7185
Epoch 179/200
120/120 [==============================] - ETA: 0s - loss: 0.4694 - accuracy: 0.8625
Epoch 179: val_loss did not improve from 0.59550
120/120 [==============================] - 1s 5ms/step - loss: 0.4694 - accuracy: 0.8625 - val_loss: 0.6872 - val_accuracy: 0.7311
Epoch 180/200
107/120 [=========

## k-fold Training evaluation

In [19]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.742226,0.749111,0.819208,0.728796,0.755642,0.484679
Train,0.800798,0.809187,0.870712,0.787153,0.814440,0.601873


In [20]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.800000,0.809989,"[0.0, 0.0010504201680672268, 0.065126050420168...","[0.0, 0.0, 0.0, 0.001049317943336831, 0.001049...","[1.9755945, 0.9755946, 0.9075918, 0.907419, 0....",0.872236,0.783613,0.816369,0.600311
1,0,Test,0.744235,0.751073,"[0.0, 0.0041841004184100415, 0.016736401673640...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9597645, 0.95976454, 0.92439914, 0.9196951,...",0.804894,0.732218,0.756303,0.488649
2,1,Train,0.803675,0.805702,"[0.0, 0.001049317943336831, 0.0325288562434417...","[0.0, 0.0, 0.0, 0.0010504201680672268, 0.00105...","[1.9765937, 0.97659373, 0.9182238, 0.91768277,...",0.865747,0.800630,0.806723,0.607362
3,1,Test,0.758910,0.773333,"[0.0, 0.004201680672268907, 0.0504201680672268...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[1.9609697, 0.9609697, 0.89984107, 0.89979213,...",0.819002,0.731092,0.786611,0.518534
4,2,Train,0.785939,0.799122,"[0.0, 0.001049317943336831, 0.0367261280167890...","[0.0, 0.0, 0.0, 0.001049317943336831, 0.001049...","[1.9668239, 0.96682394, 0.9027783, 0.9025074, ...",0.863883,0.763903,0.807975,0.572434
5,2,Test,0.741597,0.746781,"[0.0, 0.004201680672268907, 0.0672268907563025...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9361916, 0.9361915, 0.8874337, 0.88181424, ...",0.819610,0.731092,0.752101,0.483300
6,3,Train,0.816894,0.821962,"[0.0, 0.001049317943336831, 0.0472193074501574...","[0.0, 0.0, 0.0, 0.001049317943336831, 0.001049...","[1.9737983, 0.9737983, 0.92189777, 0.92109084,...",0.881473,0.809024,0.824764,0.633867
7,3,Test,0.733193,0.738197,"[0.0, 0.004201680672268907, 0.0462184873949579...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9838591, 0.9838591, 0.9297501, 0.9297028, 0...",0.830273,0.722689,0.743697,0.466490
8,4,Train,0.797482,0.809160,"[0.0, 0.001049317943336831, 0.0566631689401888...","[0.0, 0.0, 0.0, 0.001049317943336831, 0.001049...","[1.978077, 0.97807705, 0.9072833, 0.9071587, 0...",0.870218,0.778594,0.816369,0.595388
9,4,Test,0.733193,0.736170,"[0.0, 0.004201680672268907, 0.0546218487394958...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9386165, 0.9386166, 0.905668, 0.90517706, 0...",0.822258,0.726891,0.739496,0.466424


## Using k-fold Models

### Performance of each k-fold model

In [21]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.611755,0.241282,0.655327,0.625616,0.609002,0.176289


In [22]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.617959,0.241715,"[0.0, 0.0049261083743842365, 0.014778325123152...","[0.0, 0.0, 0.0, 0.0019569471624266144, 0.00195...","[1.9737401, 0.97374004, 0.9657273, 0.95779574,...",0.653553,0.610837,0.619374,0.173501
1,1,Independent,0.599184,0.235294,"[0.0, 0.0049261083743842365, 0.014778325123152...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.975141, 0.97514105, 0.96834344, 0.9609697, ...",0.649273,0.630542,0.592955,0.167252
2,2,Independent,0.606531,0.234286,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.9660288, 0.9660288, 0.9577928, 0.9546889, 0...",0.655009,0.605911,0.606654,0.159712
3,3,Independent,0.605714,0.242647,"[0.0, 0.0049261083743842365, 0.014778325123152...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.9836713, 0.9836713, 0.97563213, 0.9707484, ...",0.663897,0.650246,0.596869,0.184927
4,4,Independent,0.629388,0.252465,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.9745195, 0.97451955, 0.96712923, 0.9669862,...",0.654902,0.630542,0.629159,0.196056


### Mean score with k-fold models

In [23]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.609796,0.241055,0.657669,0.630542,0.605675,0.177238


### Voting score with k-fold models

In [24]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.616327,0.244742,0.62945,0.630542,0.613503,0.183453


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [25]:
model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = monitor, verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

del model
tf.keras.backend.clear_session()

model = tf.keras.models.load_model(current_model_path)

Epoch 1/200
138/149 [==========================>...] - ETA: 0s - loss: 0.9909 - accuracy: 0.5408
Epoch 1: val_loss improved from inf to 0.96455, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KmerRFE\5fold\models\_fullModel.hdf5
149/149 [==============================] - 1s 7ms/step - loss: 0.9889 - accuracy: 0.5416 - val_loss: 0.9646 - val_accuracy: 0.3461
Epoch 2/200
144/149 [===========================>..] - ETA: 0s - loss: 0.9356 - accuracy: 0.5530
Epoch 2: val_loss improved from 0.96455 to 0.91514, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KmerRFE\5fold\models\_fullModel.hdf5
149/149 [==============================] - 1s 6ms/step - loss: 0.9347 - accuracy: 0.5558 - val_loss: 0.9151 - val_accuracy: 0.4808
Epoch 3/200
139/149 [==========================>...] - ETA: 0s - loss: 0.8931 - accuracy: 0.5594
Epoch 3: val_loss improved from 0.91514 to 0.87810, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KmerRFE\5fold\models\_fullModel.hdf5
149/

Epoch 27/200
145/149 [============================>.] - ETA: 0s - loss: 0.5754 - accuracy: 0.7776
Epoch 27: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5750 - accuracy: 0.7771 - val_loss: 0.8030 - val_accuracy: 0.6082
Epoch 28/200
147/149 [============================>.] - ETA: 0s - loss: 0.5788 - accuracy: 0.7785
Epoch 28: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5794 - accuracy: 0.7783 - val_loss: 0.8318 - val_accuracy: 0.5902
Epoch 29/200
140/149 [===========================>..] - ETA: 0s - loss: 0.5769 - accuracy: 0.7844
Epoch 29: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 6ms/step - loss: 0.5750 - accuracy: 0.7855 - val_loss: 0.7955 - val_accuracy: 0.6204
Epoch 30/200
149/149 [==============================] - ETA: 0s - loss: 0.5702 - accuracy: 0.7746
Epoch 30: val_loss did not improve from 0.73422
149/149 [==============

Epoch 57/200
147/149 [============================>.] - ETA: 0s - loss: 0.5304 - accuracy: 0.8151
Epoch 57: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5310 - accuracy: 0.8153 - val_loss: 0.8247 - val_accuracy: 0.6269
Epoch 58/200
148/149 [============================>.] - ETA: 0s - loss: 0.5243 - accuracy: 0.8201
Epoch 58: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5260 - accuracy: 0.8186 - val_loss: 0.8911 - val_accuracy: 0.5869
Epoch 59/200
143/149 [===========================>..] - ETA: 0s - loss: 0.5273 - accuracy: 0.8094
Epoch 59: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5268 - accuracy: 0.8119 - val_loss: 0.8276 - val_accuracy: 0.6278
Epoch 60/200
144/149 [===========================>..] - ETA: 0s - loss: 0.5293 - accuracy: 0.8134
Epoch 60: val_loss did not improve from 0.73422
149/149 [==============

Epoch 87/200
148/149 [============================>.] - ETA: 0s - loss: 0.5178 - accuracy: 0.8264
Epoch 87: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5175 - accuracy: 0.8266 - val_loss: 0.8747 - val_accuracy: 0.6016
Epoch 88/200
147/149 [============================>.] - ETA: 0s - loss: 0.5222 - accuracy: 0.8231
Epoch 88: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5205 - accuracy: 0.8245 - val_loss: 0.8585 - val_accuracy: 0.6098
Epoch 89/200
146/149 [============================>.] - ETA: 0s - loss: 0.5193 - accuracy: 0.8266
Epoch 89: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5189 - accuracy: 0.8262 - val_loss: 0.8633 - val_accuracy: 0.6073
Epoch 90/200
136/149 [==========================>...] - ETA: 0s - loss: 0.5097 - accuracy: 0.8217
Epoch 90: val_loss did not improve from 0.73422
149/149 [==============

149/149 [==============================] - 1s 5ms/step - loss: 0.5042 - accuracy: 0.8426 - val_loss: 0.8713 - val_accuracy: 0.6278
Epoch 117/200
148/149 [============================>.] - ETA: 0s - loss: 0.5013 - accuracy: 0.8345
Epoch 117: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5011 - accuracy: 0.8346 - val_loss: 0.8607 - val_accuracy: 0.6335
Epoch 118/200
148/149 [============================>.] - ETA: 0s - loss: 0.5008 - accuracy: 0.8340
Epoch 118: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5003 - accuracy: 0.8342 - val_loss: 0.8203 - val_accuracy: 0.6482
Epoch 119/200
146/149 [============================>.] - ETA: 0s - loss: 0.5038 - accuracy: 0.8386
Epoch 119: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 5ms/step - loss: 0.5043 - accuracy: 0.8380 - val_loss: 0.9271 - val_accuracy: 0.5918
Epoch 120/200
145/149 [=========

Epoch 146/200
147/149 [============================>.] - ETA: 0s - loss: 0.5013 - accuracy: 0.8372
Epoch 146: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 4ms/step - loss: 0.5009 - accuracy: 0.8375 - val_loss: 0.8784 - val_accuracy: 0.6351
Epoch 147/200
139/149 [==========================>...] - ETA: 0s - loss: 0.4928 - accuracy: 0.8426
Epoch 147: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 4ms/step - loss: 0.5005 - accuracy: 0.8401 - val_loss: 0.8747 - val_accuracy: 0.6343
Epoch 148/200
138/149 [==========================>...] - ETA: 0s - loss: 0.4856 - accuracy: 0.8456
Epoch 148: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 4ms/step - loss: 0.4864 - accuracy: 0.8459 - val_loss: 0.9427 - val_accuracy: 0.6008
Epoch 149/200
135/149 [==========================>...] - ETA: 0s - loss: 0.4955 - accuracy: 0.8468
Epoch 149: val_loss did not improve from 0.73422
149/149 [======

149/149 [==============================] - 1s 4ms/step - loss: 0.4927 - accuracy: 0.8476 - val_loss: 0.9184 - val_accuracy: 0.6212
Epoch 176/200
136/149 [==========================>...] - ETA: 0s - loss: 0.4829 - accuracy: 0.8456
Epoch 176: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 4ms/step - loss: 0.4841 - accuracy: 0.8430 - val_loss: 0.8813 - val_accuracy: 0.6367
Epoch 177/200
138/149 [==========================>...] - ETA: 0s - loss: 0.4936 - accuracy: 0.8533
Epoch 177: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 4ms/step - loss: 0.4913 - accuracy: 0.8539 - val_loss: 0.9078 - val_accuracy: 0.6269
Epoch 178/200
141/149 [===========================>..] - ETA: 0s - loss: 0.4928 - accuracy: 0.8440
Epoch 178: val_loss did not improve from 0.73422
149/149 [==============================] - 1s 4ms/step - loss: 0.4928 - accuracy: 0.8417 - val_loss: 0.8849 - val_accuracy: 0.6400
Epoch 179/200
140/149 [=========

In [26]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

del model
tf.keras.backend.clear_session()

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.627755,0.242363,0.646805,0.586207,0.636008,0.1686


In [27]:
print(classification_report(indpe_labels, np.round(y_pred).astype(int)))

              precision    recall  f1-score   support

           0       0.89      0.64      0.74      1022
           1       0.24      0.59      0.34       203

    accuracy                           0.63      1225
   macro avg       0.56      0.61      0.54      1225
weighted avg       0.78      0.63      0.67      1225

